In [ ]:
key = 'ZGvYmsi4yifshynDqm0fN89tW3PT%2F6bwWgJKimuqSn%2BI3qIew%2BXTrJDgCwE7m8ASq%2BElizrpQrnh%2Bi%2FYz5zEFQ%3D%3D'
normal_key = 'ZGvYmsi4yifshynDqm0fN89tW3PT/6bwWgJKimuqSn+I3qIew+XTrJDgCwE7m8ASq+ElizrpQrnh+i/Yz5zEFQ=='

In [22]:
import datetime
import urllib.request
import time
import json
import pandas as pd
import matplotlib

In [23]:

def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s " % (datetime.datetime.now()),url )
        return None

In [24]:
def getToursimStatsItem(yyyymm, nat_cd, ed_cd):
    service_url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
    parameters = "?_type=json&serviceKey="+key
    parameters += "&YM="+yyyymm
    parameters += "&NAT_CD=" + nat_cd
    parameters += "&ED_CD="+ed_cd

    url = service_url+parameters
    print(url) #url 체크

    responseDecode = getRequestUrl(url)

    if responseDecode == None:
        return None
    else:
        return json.loads(responseDecode)

print(getToursimStatsItem('2017', '112', '2024')['response'])

http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?_type=json&serviceKey=ZGvYmsi4yifshynDqm0fN89tW3PT%2F6bwWgJKimuqSn%2BI3qIew%2BXTrJDgCwE7m8ASq%2BElizrpQrnh%2Bi%2FYz5zEFQ%3D%3D&YM=2017&NAT_CD=112&ED_CD=2024
[2025-09-24 17:53:53.470333] Url Request Success
{'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': '', 'numOfRows': 10, 'pageNo': 1, 'totalCount': 0}}


In [33]:
# 
def getToursimStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []

    natName = ''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12))
    isDataEnd = 0
    for year in range(nStartYear, nEndYear+1):
        for month in range(1,13):
            if(isDataEnd == 1):
                break
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getToursimStatsItem(yyyymm, nat_cd, ed_cd)
            if (jsonData['response']['header']['resultMsg'] == 'OK'):
                if jsonData['response']['body']['items'] == '':
                    isDataEnd = 1
                    if (month - 1) == 0:
                        year = year - 1
                        month = 13
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음")
                    jsonData = getToursimStatsItem(dataEND, nat_cd, ed_cd)
                    break
            natName = jsonData['response']['body']['items']['natKorNm']
            natName = natName.replace(' ','')
            num = jsonData['response']['body']['items']['item']['num']
            ed = jsonData['response']['body']['items']['ed']
            print ("[%s %s %s]" %(natName, yyyymm, num))
            print('-'*70)
            jsonResult.append({'nat_name':natName, 'nat_cd':nat_cd, 'yyyymm':yyyymm, 'visit_cnt':num})
            result.append([natName, nat_cd, yyyymm, num])
    return (jsonResult, result, natName, ed, dataEND)



In [36]:
def main():
    jsonResult = []
    result = []
    natName = ''

    print("<<국내 입국 데이터 조회>>")
    nat_cd = input('국가 코드 입력, 중국:112, 일본:130, 미국:275')
    nStartYear = int(input('데이터를 몇 년부터 수집할까요?'))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요?'))

    ed_cd = 'E'
    jsonResult, result, natName, ed, dataEND = getToursimStatsService(nat_cd, ed_cd, nStartYear, nEndYear)

    if (natName == ""):
        print("데이터 전달이 안됨")
    else:
        with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND), 'w', encoding='utf8') as outfile:
            jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascil = False)
            outfile.write(jsonFile)
        colums = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
        result_df = pd.DataFrame(result, colums = colums)
        result_df.to_csv('./%s_%s_%d_%s.csv' %(natName, ed, nStartYear, dataEND), index = False, encoding = 'cp949')



In [35]:
if __name__ == "__main__":
    main()

<<국내 입국 데이터 조회>>
http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?_type=json&serviceKey=ZGvYmsi4yifshynDqm0fN89tW3PT%2F6bwWgJKimuqSn%2BI3qIew%2BXTrJDgCwE7m8ASq%2BElizrpQrnh%2Bi%2FYz5zEFQ%3D%3D&YM=201701&NAT_CD=112&ED_CD=E
[2025-09-24 18:00:16.415368] Url Request Success


KeyError: 'natKorNm'

In [29]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager,rc

visitCnt = []
visitVM = []

index = []
i = 0

for item in jsonResult:
    index.append(i)
    visitCnt.append(item['visit_cnt'])
    visitVM.append(item['yyyymm'])
    i += 1

plt.rc('font', family = 'Malgum Gothic')

plt.xticks(idnex, visitYM)
plt.plot(index, visitCnt)
plt.xlabel('방문월')
plt.xticks(rotation=45)
plt.ylabel(natName = '에서 온 방문객수')
plt.grid(True)
plt.show()

NameError: name 'jsonResult' is not defined